# Investigating the medical appointment no-show trends
The following dataset collects information from 100,000 medical appointments taking place over three spring months of 2016 in Brazil. The data set is focused on the attendance of patients and their characteristics, such as gender, age, date of appointment, health conditions, medical scholarship, SMS reminders, etc. A number of characteristics of the patient are included in each row.

In the following report I will analyse several factors by fitting them into a classification ML model and determine whether those may affect attendance of patients to their scheduled medical appointments:

- Age
- Long-term health condition or disability
- Medical Scholarship
- Waiting time for scheduled appointment if it was booked in advance
- SMS reminders

In [1]:
# libraries
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline
# Logistic regression class
from sklearn.linear_model import LogisticRegression


In [2]:
# load the data
df = pd.read_csv("clean_nsa.csv")
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,No_show,No_show_b,Waiting_time,Waiting_period,Age_group
0,2.987250e+13,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,No,0,0,Day,Adult
1,5.589978e+14,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,No,0,0,Day,Adult
2,4.262962e+12,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,No,0,0,Day,Adult
3,8.679512e+11,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,0,0,Day,Child
4,8.841186e+12,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,No,0,0,Day,Adult


In [3]:
df.tail()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,No_show,No_show_b,Waiting_time,Waiting_period,Age_group
110516,2.572134e+12,5651768,F,2016-05-03,2016-06-07,56,MARIA ORTIZ,0,0,0,0,0,1,No,0,35,Trimester,Adult
110517,3.596266e+12,5650093,F,2016-05-03,2016-06-07,51,MARIA ORTIZ,0,0,0,0,0,1,No,0,35,Trimester,Adult
110518,1.557663e+13,5630692,F,2016-04-27,2016-06-07,21,MARIA ORTIZ,0,0,0,0,0,1,No,0,41,Trimester,Young adult
110519,9.213493e+13,5630323,F,2016-04-27,2016-06-07,38,MARIA ORTIZ,0,0,0,0,0,1,No,0,41,Trimester,Adult
110520,3.775115e+14,5629448,F,2016-04-27,2016-06-07,54,MARIA ORTIZ,0,0,0,0,0,1,No,0,41,Trimester,Adult


In [4]:
df.describe()

,PatientId,AppointmentID,Age,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,No_show_b,Waiting_time
count,1.105210e+05,1.105210e+05,110521.000000,110521.000000,110521.000000,110521.000000,110521.000000,110521.000000,110521.000000,110521.000000,110521.000000
mean,1.474906e+14,5.675304e+06,37.089386,0.098271,0.197257,0.071869,0.030401,0.020259,0.321043,0.201898,10.184345
std,2.560860e+14,7.129691e+04,23.109885,0.297682,0.397929,0.258272,0.171690,0.140884,0.466879,0.401419,15.255153
min,3.921784e+04,5.030230e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172457e+12,5.640284e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173185e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
75%,9.438963e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,15.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,179.000000


In [5]:
# remove no_show, leaving only no_show_b, as this will be usen in a model.
# No_show_b represents 0 = No, 1 = Yes to missed appointments.
df = df.drop(columns=['No_show'], axis=1, inplace = True)

### EDA Summary
According to this particular data set, there are following conclusions:
- Certain age groups such as Teenagers and Young adults may be considered to be a factor to the probability of no-shows.
- A long-term health condition or disability does not have a big effect on the probability of medical no-shows in this dataset.
- Medical Scholarship yielded interesting results, as people with scholarships tend to miss their appointments more, according to this dataset.
- Waiting time for scheduled appointment does have an effect on the probability of attendance, the more time passes between booking date and appointment, the fewer people show up.
- SMS reminders do have a slight effect on attendance, however, with time they become less effective.

# Logistic regression
Since we are dealing with just two possibilities, this is a binary classification problem. I propose to used Logistic regression model to estimate possibility of medical appointment no-show.

In [ ]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X, y)

# predict the response values for the observations in X
logreg.predict(X)